In [ ]:
!pip install -q amplpy ampltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.3 MB/s eta 0:00:00


In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [ ]:
from pickle import TRUE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# File path to your Excel file
file_path = '/content/drive/MyDrive/INFO 645/realignment_data_ampl.xlsx'

### Load Data ###

# Load Sheet1: Hours required annually
sheet1_data = pd.read_excel(file_path, sheet_name="Sheet1", header=0)
S = list(sheet1_data.iloc[:, 0])  # Stores
A = sheet1_data.columns[1:].tolist()  # Areas
hours_required = sheet1_data.set_index(sheet1_data.columns[0]).to_dict(orient='index')
print("Hours Required Dictionary Sample:", dict(list(hours_required.items())[:5]))


Hours Required Dictionary Sample: {'Albemarle_County': {'Inventory': 460, 'Payroll': 771, 'Hiring': 0, 'Marketing': 96, 'Merchandising': 90}, 'Amherst_County': {'Inventory': 2, 'Payroll': 28, 'Hiring': 0, 'Marketing': 8, 'Merchandising': 0}, 'Augusta_County': {'Inventory': 341, 'Payroll': 737, 'Hiring': 2806, 'Marketing': 260, 'Merchandising': 65}, 'Buckingham_County': {'Inventory': 46, 'Payroll': 40, 'Hiring': 408, 'Marketing': 84, 'Merchandising': 59}, 'Caroline_County': {'Inventory': 13, 'Payroll': 136, 'Hiring': 170, 'Marketing': 10, 'Merchandising': 55}}


In [ ]:
# Load Sheet3: Hours available
sheet3_data = pd.read_excel(file_path, sheet_name="Sheet3", header=0)
R = list(sheet3_data.iloc[:, 0])  # Regions
hours_available = sheet3_data.set_index(sheet3_data.columns[0]).to_dict(orient='index')
print("Hours Available Dictionary Sample:", dict(list(hours_available.items())[:5]))

Hours Available Dictionary Sample: {'Richmond': {'Inventory': 5000, 'Payroll': 3025, 'Hiring': 1225, 'Marketing': 1750, 'Merchandising': 3675}, 'Tappahannock': {'Inventory': 3400, 'Payroll': 5550, 'Hiring': 3250, 'Marketing': 1200, 'Merchandising': 1600}, 'Warrenton': {'Inventory': 825, 'Payroll': 2500, 'Hiring': 3375, 'Marketing': 1325, 'Merchandising': 850}, 'Staunton': {'Inventory': 3550, 'Payroll': 3450, 'Hiring': 9100, 'Marketing': 1700, 'Merchandising': 1850}}


In [ ]:
# Load mileage data
sheet4_data = pd.read_excel(file_path, sheet_name="Sheet4", header=0)

# Replace '--' or any non-numeric value with 1000
sheet4_data.replace({'--': '1000'}, inplace=True)

# Remove commas and ensure consistent formatting
sheet4_data.iloc[:, 1:] = sheet4_data.iloc[:, 1:].replace({',': ''}, regex=True)

# Convert all columns except the first (stores) to numeric, coercing errors
sheet4_data.iloc[:, 1:] = sheet4_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Fill any remaining NaN values with 1000 (e.g., for cells that failed conversion)
sheet4_data.fillna(1000, inplace=True)

# Extract regions (all column headers after the first one)
regions = sheet4_data.columns[1:].tolist()

# Create the mileage dictionary
mileage_dict = {
    row.iloc[0]: {region: row[region] for region in regions}
    for _, row in sheet4_data.iterrows()
}

# Debug: Verify the structure of mileage_dict
print("Extracted Regions:", regions)
print("Mileage Dictionary Sample:\n", dict(list(mileage_dict.items())[:5]))


print("Mileage Dictionary Sample:", dict(list(mileage_dict.items())[:5]))

Extracted Regions: ['Staunton', 'Warrenton', 'Richmond', 'Tappahannock']
Mileage Dictionary Sample:
 {'Albemarle_County': {'Staunton': 37, 'Warrenton': 71, 'Richmond': 70, 'Tappahannock': 110}, 'Amherst_County': {'Staunton': 56, 'Warrenton': 1000, 'Richmond': 120, 'Tappahannock': 1000}, 'Augusta_County': {'Staunton': 0, 'Warrenton': 121, 'Richmond': 1000, 'Tappahannock': 1000}, 'Buckingham_County': {'Staunton': 66, 'Warrenton': 108, 'Richmond': 72, 'Tappahannock': 1000}, 'Caroline_County': {'Staunton': 1000, 'Warrenton': 62, 'Richmond': 42, 'Tappahannock': 38}}
Mileage Dictionary Sample: {'Albemarle_County': {'Staunton': 37, 'Warrenton': 71, 'Richmond': 70, 'Tappahannock': 110}, 'Amherst_County': {'Staunton': 56, 'Warrenton': 1000, 'Richmond': 120, 'Tappahannock': 1000}, 'Augusta_County': {'Staunton': 0, 'Warrenton': 121, 'Richmond': 1000, 'Tappahannock': 1000}, 'Buckingham_County': {'Staunton': 66, 'Warrenton': 108, 'Richmond': 72, 'Tappahannock': 1000}, 'Caroline_County': {'Staunton'

In [ ]:
# Load Sheet5: Travel time data
sheet5_data = pd.read_excel(file_path, sheet_name="Sheet5", header=0)
sheet5_data.replace('--', 100, inplace=True)
sheet5_data.iloc[:, 1:] = sheet5_data.iloc[:, 1:].apply(pd.to_numeric)
travel_time_dict = {
    row.iloc[0]: {region: row[region] for region in R}
    for _, row in sheet5_data.iterrows()
}
print("Travel Time Dictionary Sample:", dict(list(travel_time_dict.items())[:5]))

Travel Time Dictionary Sample: {'Albemarle_County': {'Richmond': 1.22, 'Tappahannock': 2.0, 'Warrenton': 1.56, 'Staunton': 0.66}, 'Amherst_County': {'Richmond': 2.1, 'Tappahannock': 100.0, 'Warrenton': 100.0, 'Staunton': 1.08}, 'Augusta_County': {'Richmond': 100.0, 'Tappahannock': 100.0, 'Warrenton': 2.06, 'Staunton': 0.0}, 'Buckingham_County': {'Richmond': 1.55, 'Tappahannock': 100.0, 'Warrenton': 2.43, 'Staunton': 1.3}, 'Caroline_County': {'Richmond': 0.78, 'Tappahannock': 0.82, 'Warrenton': 1.45, 'Staunton': 100.0}}


<ipython-input-9-fcedc31ed266>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sheet5_data.replace('--', 100, inplace=True)


In [ ]:
# Load Sheet6: Round trips
sheet6_data = pd.read_excel(file_path, sheet_name="Sheet6", header=0)
round_trips = sheet6_data.set_index(sheet6_data.columns[0]).to_dict(orient='index')

### Compute Combined Hours Required ###
hours_required_combined = {}
for store in hours_required:
    for area in hours_required[store]:
        try:
            base_hours = hours_required[store][area]
            travel_hours = sum(
                round_trips[store][area] * travel_time_dict[store][region]
                for region in travel_time_dict[store]
            )
            hours_required_combined[(store, area)] = base_hours + travel_hours
        except KeyError as e:
            print(f"Missing data for {store} in area {area}: {e}")

# Flatten hours_available
hours_available_flat = {
    (region, area): hours_available[region][area]
    for region in hours_available
    for area in hours_available[region]
}

### Debugging Outputs ###
print("Stores (S):", S)
print("Areas (A):", A)
print("Regions (R):", R)
print("\nCombined Hours Required (Sample):", dict(list(hours_required_combined.items())[:5]))
print("\nFlattened Hours Available (Sample):", dict(list(hours_available_flat.items())[:5]))

Stores (S): ['Albemarle_County', 'Amherst_County', 'Augusta_County', 'Buckingham_County', 'Caroline_County', 'Charles_City_County', 'Chesterfield_County', 'City_of_Fredericksburg', 'City_of_Richmond', 'Culpeper_County', 'Cumberland_County', 'Dinwiddie_County', 'Essex_County', 'Fauquier_County', 'Fluvanna_County', 'Goochland_County', 'Greene_County', 'Hanover_County', 'Henrico_County', 'Hopewell_County', 'James_City_County', 'King_and_Queen_County', 'King_George_County', 'King_William_County', 'Louisa_County', 'Madison_County', 'Mathews_County', 'Nelson_County', 'New_Kent_County', 'Orange_County', 'Page_County', 'Powhatan_County', 'Prince_George_County', 'Prince_William_County', 'Rappahannock_County', 'Rockbridge_County', 'Rockingham_County', 'Shenandoah_County', 'Spotsylvania_County', 'Stafford_County', 'Warren_County', 'Westmoreland_County', 'York_County']
Areas (A): ['Inventory', 'Payroll', 'Hiring', 'Marketing', 'Merchandising']
Regions (R): ['Richmond', 'Tappahannock', 'Warrenton',

In [ ]:
# Prepare data for AMPL
AMPL_data = {
    'R': R,  # Regions
    'S': S,  # Stores
    'A': A,  # Areas
    'hours_required': hours_required_combined,  # Hours required by store and area
    'hours_available': hours_available_flat,  # Hours available by region and area
    'mileage': mileage_dict,  # Two-dimensional dictionary: mileage[store][region]
    'travel_time': travel_time_dict,  # Two-dimensional dictionary: travel_time[store][region]
    'round_trips': round_trips  # Total round trips for each store and area
}

# Debugging and Verification
print("\n===== Debugging Data Verification =====\n")

# Display regions
if 'R' in AMPL_data:
    print("Regions (R):", ", ".join(AMPL_data['R']))
else:
    print("Regions (R): Data not found.")

# Display stores
if 'S' in AMPL_data:
    print("\nStores (S):", ", ".join(AMPL_data['S']))
else:
    print("\nStores (S): Data not found.")

# Display areas
if 'A' in AMPL_data:
    print("\nAreas (A):", ", ".join(AMPL_data['A']))
else:
    print("\nAreas (A): Data not found.")

# Display a sample of hours_required
if 'hours_required' in AMPL_data:
    print("\nHours Required by Stores and Areas (Sample):")
    print(dict(list(AMPL_data['hours_required'].items())[:5]))
else:
    print("\nHours Required by Stores and Areas: Data not found.")

# Display a sample of hours_available
if 'hours_available' in AMPL_data:
    print("\nHours Available by Regions and Areas (Sample):")
    print(dict(list(AMPL_data['hours_available'].items())[:5]))
else:
    print("\nHours Available by Regions and Areas: Data not found.")

# Display mileage dictionary sample
if 'mileage' in AMPL_data:
    print("\nMileage Dictionary (Sample):")
    print(dict(list(AMPL_data['mileage'].items())[:5]))
else:
    print("\nMileage Dictionary: Data not found.")

# Display travel time dictionary sample
if 'travel_time' in AMPL_data:
    print("\nTravel Time Dictionary (Sample):")
    print(dict(list(AMPL_data['travel_time'].items())[:5]))
else:
    print("\nTravel Time Dictionary: Data not found.")

# Display round trips sample
if 'round_trips' in AMPL_data:
    print("\nRound Trips (Sample):")
    print(dict(list(AMPL_data['round_trips'].items())[:5]))
else:
    print("\nRound Trips: Data not found.")



===== Debugging Data Verification =====

Regions (R): Richmond, Tappahannock, Warrenton, Staunton

Stores (S): Albemarle_County, Amherst_County, Augusta_County, Buckingham_County, Caroline_County, Charles_City_County, Chesterfield_County, City_of_Fredericksburg, City_of_Richmond, Culpeper_County, Cumberland_County, Dinwiddie_County, Essex_County, Fauquier_County, Fluvanna_County, Goochland_County, Greene_County, Hanover_County, Henrico_County, Hopewell_County, James_City_County, King_and_Queen_County, King_George_County, King_William_County, Louisa_County, Madison_County, Mathews_County, Nelson_County, New_Kent_County, Orange_County, Page_County, Powhatan_County, Prince_George_County, Prince_William_County, Rappahannock_County, Rockbridge_County, Rockingham_County, Shenandoah_County, Spotsylvania_County, Stafford_County, Warren_County, Westmoreland_County, York_County

Areas (A): Inventory, Payroll, Hiring, Marketing, Merchandising

Hours Required by Stores and Areas (Sample):
{('Albe

In [ ]:
ampl.eval('''

reset;

# Sets
set R;   # Regions
set S;   # Stores
set A;   # Areas

# Parameters
param mileage_cost >= 0 default 0.585;  # Mileage cost per mile
param salary_per_hour >= 0 default 26;  # Salary cost per hour

# Data Parameters
param mileage{S, R} >= 0;      # Mileage distance from each store to each region
param travel_time{S, R} >= 0;  # Travel time from each store to each region (in hours)
param base_hours_required{S, A} >= 0;  # Base hours required annually by each store for each area
param hours_available{R, A} >= 0;      # Hours available annually for each region for each area
param round_trips{S, A} >= 0;          # Number of round trips for each store and area

# Decision Variables
var x{S, R} binary;     # Binary decision variable: 1 if store j is assigned to region i, 0 otherwise

# Objective Function
minimize TotalCost:
    sum {j in S, i in R}
        x[j, i] * (
            mileage_cost * mileage[j, i] * sum {a in A} round_trips[j, a]  # Total mileage cost
            + salary_per_hour * travel_time[j, i] * sum {a in A} round_trips[j, a]  # Total salary cost
        );

# Constraints

# 1. Each store must be assigned to exactly one region
subject to AssignEachStoreOnce {j in S}:
    sum {i in R} x[j, i] = 1;

# 2. Hours required by stores in each region for each area must not exceed available hours
subject to HoursConstraint {i in R, a in A}:
    sum {j in S} x[j, i] * (
        base_hours_required[j, a]  # Base hours from Sheet1
        + round_trips[j, a] * travel_time[j, i]  # Travel hours
    ) <= hours_available[i, a];

''')


In [ ]:
# Debugging and Verification
print("\n===== Debugging Data Verification =====\n")

# Display the list of regions
print("Regions (R):")
print(AMPL_data['R'])

# Display the list of stores
print("\nStores (S):")
print(AMPL_data['S'])

# Display hours required (showing only the first 5 entries for brevity)
print("\nHours Required (Sample):")
for key, value in list(AMPL_data['hours_required'].items())[:5]:
    print(f"Store {key[0]}, Area {key[1]}: {value}")

# Display hours available (showing only the first 5 entries for brevity)
print("\nHours Available (Sample):")
for key, value in list(AMPL_data['hours_available'].items())[:5]:
    print(f"Region {key[0]}, Area {key[1]}: {value}")

# Display travel time for the first store
print("\nTravel Time (First Store):")
first_store = list(AMPL_data['travel_time'].keys())[0]
print(f"Store: {first_store}, Travel Times: {AMPL_data['travel_time'][first_store]}")

# Display round trips
print("\nRound Trips (Sample):")
for key, value in list(AMPL_data['round_trips'].items())[:5]:
    print(f"Store {key[0]}, Area {key[1]}: {value}")

# Display mileage for the first store
print("\nMileage (First Store):")
first_store = list(AMPL_data['mileage'].keys())[0]
print(f"Store: {first_store}, Mileage: {AMPL_data['mileage'][first_store]}")




===== Debugging Data Verification =====

Regions (R):
['Richmond', 'Tappahannock', 'Warrenton', 'Staunton']

Stores (S):
['Albemarle_County', 'Amherst_County', 'Augusta_County', 'Buckingham_County', 'Caroline_County', 'Charles_City_County', 'Chesterfield_County', 'City_of_Fredericksburg', 'City_of_Richmond', 'Culpeper_County', 'Cumberland_County', 'Dinwiddie_County', 'Essex_County', 'Fauquier_County', 'Fluvanna_County', 'Goochland_County', 'Greene_County', 'Hanover_County', 'Henrico_County', 'Hopewell_County', 'James_City_County', 'King_and_Queen_County', 'King_George_County', 'King_William_County', 'Louisa_County', 'Madison_County', 'Mathews_County', 'Nelson_County', 'New_Kent_County', 'Orange_County', 'Page_County', 'Powhatan_County', 'Prince_George_County', 'Prince_William_County', 'Rappahannock_County', 'Rockbridge_County', 'Rockingham_County', 'Shenandoah_County', 'Spotsylvania_County', 'Stafford_County', 'Warren_County', 'Westmoreland_County', 'York_County']

Hours Required (Sam

In [ ]:
# Assign sets
ampl.set['R'] = R  # Regions
ampl.set['S'] = S  # Stores
ampl.set['A'] = A  # Areas

# Assign parameters
# Mileage: Flatten the nested dictionary to pass to AMPL
ampl.param['mileage'] = {
    (store, region): mileage_dict[store][region]
    for store in S  # Ensure only stores in S are included
    for region in R  # Ensure only regions in R are included
}

# Travel Time: Flatten the nested dictionary to pass to AMPL
ampl.param['travel_time'] = {
    (store, region): travel_time_dict[store][region]
    for store in S  # Ensure only stores in S are included
    for region in R  # Ensure only regions in R are included
}

# Round Trips: Flatten round trips (area-specific)
round_trips_flat = {
    (store, area): round_trips[store][area]
    for store in S  # Ensure only stores in S are included
    for area in A  # Ensure only areas in A are included
}
ampl.param['round_trips'] = round_trips_flat

# Base Hours Required: Flatten base hours (from Sheet1) to handle store x area structure
base_hours_required_flat = {
    (store, area): hours_required[store][area]
    for store in S  # Ensure only stores in S are included
    for area in A  # Ensure only areas in A are included
}
ampl.param['base_hours_required'] = base_hours_required_flat

# Hours Available: Flatten to handle region x area structure
hours_available_flat = {
    (region, area): hours_available[region][area]
    for region in R  # Ensure only regions in R are included
    for area in A  # Ensure only areas in A are included
}
ampl.param['hours_available'] = hours_available_flat




In [ ]:

# Expand and display variables
ampl.eval('expand x;')

# Expand and display constraints
ampl.eval('expand AssignEachStoreOnce;')
ampl.eval('expand HoursConstraint;')



Coefficients of x['Albemarle_County','Richmond']:
	AssignEachStoreOnce['Albemarle_County']          1
	HoursConstraint['Richmond','Inventory']        555.16
	HoursConstraint['Richmond','Payroll']         1095.52
	HoursConstraint['Richmond','Marketing']        142.36
	HoursConstraint['Richmond','Merchandising']    102.2
	TotalCost                                    28486.6

Coefficients of x['Albemarle_County','Tappahannock']:
	AssignEachStoreOnce['Albemarle_County']              1
	HoursConstraint['Tappahannock','Inventory']        616
	HoursConstraint['Tappahannock','Payroll']         1303
	HoursConstraint['Tappahannock','Marketing']        172
	HoursConstraint['Tappahannock','Merchandising']    110
	TotalCost                                        45609.2

Coefficients of x['Albemarle_County','Warrenton']:
	AssignEachStoreOnce['Albemarle_County']           1
	HoursConstraint['Warrenton','Inventory']        581.68
	HoursConstraint['Warrenton','Payroll']         1185.96
	HoursConstrain

In [ ]:
ampl.setOption('solver', 'cplex')
ampl.solve()

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 195479.31
17 simplex iterations
absmipgap=2.91038e-11, relmipgap=0


In [ ]:
# Print the objective function value (Total Cost)
obj = ampl.getObjective('TotalCost')
print("\n===== Optimization Results =====\n")
print(f"Total Cost is: {obj.get().value():.2f}\n")  # Format to 2 decimal places

# Display the values of decision variables (store-to-region assignments)
print("Store-to-Region Assignments (Decision Variables):")
ampl.display('x')

# Display base hours required for each store and area
print("\nBase Hours Required by Stores and Areas:")
ampl.display('base_hours_required')

# Display available hours for each region and area
print("\nHours Available for Regions and Areas:")
ampl.display('hours_available')

# Display travel time parameter
print("\nTravel Time Parameter:")
ampl.display('travel_time')

# Display round trips parameter
print("\nRound Trips Parameter:")
ampl.display('round_trips')

# Display mileage parameter
print("\nMileage Parameter:")
ampl.display('mileage')



===== Optimization Results =====

Total Cost is: 195479.31

Store-to-Region Assignments (Decision Variables):
x [*,*]
:                      Richmond Staunton Tappahannock Warrenton    :=
Albemarle_County           0        1          0           0
Amherst_County             0        1          0           0
Augusta_County             0        1          0           0
Buckingham_County          0        1          0           0
Caroline_County            0        0          1           0
Charles_City_County        1        0          0           0
Chesterfield_County        1        0          0           0
City_of_Fredericksburg     0        0          1           0
City_of_Richmond           1        0          0           0
Culpeper_County            0        0          0           1
Cumberland_County          1        0          0           0
Dinwiddie_County           1        0          0           0
Essex_County               0        0          1           0
Fauquier_County   

In [ ]:
# Calculate and print hours assigned to each region across individual areas
print("\n===== Assigned Hours to Each Region Across Individual Areas =====\n")

# Initialize a dictionary to store the assigned hours for each region and area
assigned_hours_by_area = {region: {area: 0 for area in A} for region in R}

# Iterate over regions, areas, and stores to compute assigned hours
for region in R:
    for area in A:
        for store in S:
            # Combine base hours and travel hours based on assignment
            assigned_hours_by_area[region][area] += (
                ampl.getVariable('x').get(store, region).value() *  # Decision variable
                (
                    ampl.param['base_hours_required'][(store, area)]  # Base hours
                    + ampl.param['round_trips'][(store, area)] * ampl.param['travel_time'][(store, region)]  # Travel hours
                )
            )

# Print the results
for region, areas in assigned_hours_by_area.items():
    print(f"Region: {region}")
    for area, hours in areas.items():
        print(f"  Area: {area}, Assigned Hours: {hours:.2f}")
    print()  # Add a blank line after each region




===== Assigned Hours to Each Region Across Individual Areas =====

Region: Richmond
  Area: Inventory, Assigned Hours: 2584.80
  Area: Payroll, Assigned Hours: 2730.20
  Area: Hiring, Assigned Hours: 1097.58
  Area: Marketing, Assigned Hours: 1128.62
  Area: Merchandising, Assigned Hours: 671.40

Region: Tappahannock
  Area: Inventory, Assigned Hours: 735.24
  Area: Payroll, Assigned Hours: 943.98
  Area: Hiring, Assigned Hours: 1783.04
  Area: Marketing, Assigned Hours: 374.10
  Area: Merchandising, Assigned Hours: 708.28

Region: Warrenton
  Area: Inventory, Assigned Hours: 755.96
  Area: Payroll, Assigned Hours: 1418.44
  Area: Hiring, Assigned Hours: 1514.58
  Area: Marketing, Assigned Hours: 624.90
  Area: Merchandising, Assigned Hours: 828.78

Region: Staunton
  Area: Inventory, Assigned Hours: 1836.42
  Area: Payroll, Assigned Hours: 3399.58
  Area: Hiring, Assigned Hours: 7594.92
  Area: Marketing, Assigned Hours: 1075.22
  Area: Merchandising, Assigned Hours: 457.88

